In [24]:
# load packages
import numpy as np
import pandas as pd
# import napari
import tifffile as tf
from skimage import io # import tiff file as ndarray
import os
import matplotlib.pyplot as plt

# import custum colormaps
from colormaps import *

# import custum functions
from STP_plotting import *
from STP_processing import *

# auto refreshes imported functions if source code changes:
%load_ext autoreload
%autoreload 2



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [35]:
# choose based on run in acadia or home computer
# home_dir = "/mnt/labNAS/"
home_dir = "/Volumes/Data/"

metadata = pd.read_csv("stp_metadata.csv")

in_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/input_tifs/"
out_path = home_dir+"Emily/STP_for_MAPseq/4_python_output/output_figs/"

# add paths to metadata
for i in range(metadata.shape[0]):
    if metadata.loc[i,"species"] == "STeg":
        b_path = in_path+"norm_subtracted/STeg_220429_aligned/"+metadata.loc[i,"brain"]+"_aligned_to_STeg_220429_norm_subtracted.tif"
    elif metadata.loc[i,"species"] == "MMus":
        b_path = in_path+"norm_subtracted/MMus_220303_aligned/"+metadata.loc[i,"brain"]+"_aligned_to_MMus_220303_norm_subtracted.tif"
    
    metadata.loc[i,"path"] = b_path

# get file paths
# dir_list = os.listdir(in_path)

# metadata['path'] = dir_list

# area list from make_masks.py
areas_masked = ["grey", "CTX", "OMCc", "aud","TH", "STR", "CP", "AMY", "P", "PG", "MB", "PAG", "SCm", 
         "SNr", "HY", "CNU", "TEa", "ECT", "VISC", "AI", "GU", "BS"]


In [36]:
metadata

,brain,species,inj_site,path
0,ACC_MMus_230403_b1,MMus,ACC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
1,ACC_MMus_230404_b2,MMUs,ACC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
2,ACC_STeg_230308_b1,STeg,ACC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
3,ACC_STeg_230322_b2,STeg,ACC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
4,ACC_STeg_230501_b3,STeg,ACC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
5,OMC_MMus_220119_b0,MMus,OMC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
6,OMC_MMus_220303_b1,MMus,OMC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
7,OMC_MMus_220324_b2,MMus,OMC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
8,OMC_STeg_220208_b0,STeg,OMC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
9,OMC_STeg_220411_b1,STeg,OMC,/Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...


In [37]:
# load masks
steg_masks = []
for i in range(len(areas)):
    with open(in_path+"masks/STeg_220429_"+areas[i]+".npy", "rb") as f:
            steg_masks.append(np.load(f))

mmus_masks = []
for i in range(len(areas)):
    with open(in_path+"masks/MMus_220303_"+areas[i]+".npy", "rb") as f:
            mmus_masks.append(np.load(f))


# import images
images = []
for i in range(metadata.shape[0]):
    images.append(tf.imread(metadata.loc[i,"path"]))

# import atlases
st_atlas_path = in_path+"Steg_220429_registered_atlas_RESIZED.tif"
st_atlas = tf.imread(st_atlas_path)

mm_atlas_path = in_path+"MMus_220303_registered_atlas_RESIZED.tif"
mm_atlas = tf.imread(mm_atlas_path)


# quantify integrated fluorescence




In [39]:
# areas that match w/ mapseq data
areas_plot = ["OMCc", "CP", "aud", "AMY", "TH", "HY", "SNr", "SCm", "PG", "PAG", "BS"]


In [42]:
mmus_masks[0].shape

(212, 554, 682)

## OMC

In [38]:
# set metadata
meta_omc = metadata[metadata["inj_site"]=="OMC"]
meta_omc = meta_omc.reset_index()

images_omc = [images[i] for i in meta_omc["index"]]

In [43]:
test = images[0]
test2 = mmus_masks[0]
print(test.dtype)
print(test2.dtype)

uint16
bool


In [22]:
help(calc_fluor)

Help on function calc_fluor in module STP_processing:

calc_fluor(images, metadata, mm_masks, st_masks, mask_areas, areas_to_plot)
    Calculates integrated fluorescence on a per area basis.
    Returns a pandas dataframe
    
    Args:
        images (list): list of images to calculate
        metadata (dataframe): Pandas dataframe where each row corresponds to image in iamges
        mm_masks (list): mmus masks from mmus aligned atlas
        st_masks (list): steg masks from steg aligned atlas
        mask_areas (list): List of strings that label each mask in mm/st_masks
        areas_to_plot (list): List of strings for each area to plot



In [48]:
print(len(areas))
print(len(areas_plot))
print(len(mmus_masks))
print(len(steg_masks))
for area in areas_plot:
    print(areas.index(area))

16
11
16
16


ValueError: 'OMCc' is not in list

In [31]:
for i in range(len(images)):
    print(images[i].shape)
metadata

(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)
(201, 522, 692)


,brain,species,inj_site,path
0,ACC_MMus_230403_b1,MMus,ACC,ACC_MMus_230403_b1_aligned_to_STeg_220429_norm...
1,ACC_MMus_230404_b2,MMUs,ACC,ACC_MMus_230404_b2_aligned_to_STeg_220429_norm...
2,ACC_STeg_230308_b1,STeg,ACC,ACC_STeg_230308_b1_aligned_to_STeg_220429_norm...
3,ACC_STeg_230322_b2,STeg,ACC,ACC_STeg_230322_b2_aligned_to_STeg_220429_norm...
4,ACC_STeg_230501_b3,STeg,ACC,ACC_STeg_230501_b3_aligned_to_STeg_220429_norm...
5,OMC_MMus_220119_b0,MMus,OMC,OMC_MMus_220119_b0_aligned_to_STeg_220429_norm...
6,OMC_MMus_220303_b1,MMus,OMC,OMC_MMus_220303_b1_aligned_to_STeg_220429_norm...
7,OMC_MMus_220324_b2,MMus,OMC,OMC_MMus_220324_b2_aligned_to_STeg_220429_norm...
8,OMC_STeg_220208_b0,STeg,OMC,OMC_STeg_220208_b0_aligned_to_STeg_220429_norm...
9,OMC_STeg_220411_b1,STeg,OMC,OMC_STeg_220411_b1_aligned_to_STeg_220429_norm...


In [44]:
omc_fluor = calc_fluor(images_omc, meta_omc, mm_masks=mmus_masks, st_masks=steg_masks,
                       mask_areas=areas_masked, areas_to_plot=areas_plot)
omc_fluor

index                                                       5
brain                                      OMC_MMus_220119_b0
species                                                  MMus
inj_site                                                  OMC
path        /Volumes/Data/Emily/STP_for_MAPseq/4_python_ou...
Name: 0, dtype: object
OMCc 2
CP 6
aud 3
AMY 7
TH 4
HY 14
SNr 13
SCm 12
PG 9
PAG 11
BS 21


IndexError: list index out of range

In [26]:
row_met = metadata.iloc[i,:]
row_met['species']


'STeg'

## ACC